In [1]:
import pandas as pd
import numpy as np 
from multiprocessing import Pool
from tqdm import tqdm
import os
import torch 
import warnings
import json 
import torch.utils.data as data_utils
from torchvision import datasets, transforms
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)
pd.set_option("display.precision", 1)

/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [2]:
raw_df = pd.read_pickle("data/property_structured_12162024.pkl")
raw_df.head(2)

,address,harlink,mp_features,agent,price,posted_days,status,zipcode,num_beds,num_bath,num_half_bath,building_sqft,has_loft,num_stories,num_parking_space,address_key,date,json_path,time_epoch,image_path,json_size,image_cnt,LAT,LON,building_sqft2,Lot Size,Year Built,Garage Number,Bedrooms,Baths,Maintenance Fee,Tax Rate,tax_rate2,Recent Market Value,Recent Tax Value,Property Type,County,Private Pool,Area Pool,Private_pool_desc,Pool_feature_multiclass,Foundation_multiclass,Garage Types_multiclass,Roof Type_multiclass,floor_type_multiclass,Exterior Type_multiclass,Exterior_multiclass,Style_multiclass,finance_option_multiclass,high_school_star,middle_school_star,elementary_school_star,school_org,general_desc
57,"4513 Refugio Dr, Plano, TX 75024",https://www.har.com//homedetail/4513-refugio-d...,"4 bedrooms 3,395 Sqft. ($233/Sqft.) 3 full & 1...",Karla Davis Fathom Realty,789990.0,66.0,Under Contract - P,75024,4.0,0.0,0.0,3395.0,0.0,0.0,0.0,"4513 Refugio Dr, Plano, TX 75024",2023_11_05,/home/user/DataCenter/HAR_data/address_fullinf...,1690411643,/home/user/DataCenter/HAR_data/house_images/00...,4425,12,-96.8,33.1,3395.0,429.0,2023.0,2.0,4.0,3.5,65.1,1.9,NaN,157533.0,157533.0,single family,Collin County,No,None,None,[],[Slab],[Attached],[composition],"[carpet, ceramic tile, wood]","[brick, rock/stone]",[covered patio/porch],[traditional],[],NaN,5.0,5.0,[FRISCO ISD],"MOVE IN READY! Welcome to The Bartlet, this ge..."
8,"1516 Bay Area Blvd P12, Houston, TX 77058",https://www.har.com//homedetail/1516-bay-area-...,1 bedrooms 684 Sqft. ($117/Sqft.) 1 full baths...,Laurie Long RE/MAX Space Center,79900.0,79.0,Under Contract - PS,77058,1.0,0.0,0.0,684.0,0.0,0.0,0.0,"1516 Bay Area Blvd P12, Houston, TX 77058",2023_11_05,/home/user/DataCenter/HAR_data/address_fullinf...,1689375459,/home/user/DataCenter/HAR_data/house_images/00...,5661,11,-95.1,29.6,684.0,31348.0,1977.0,NaN,1.0,1.0,272.0,2.4,2.4,69024.0,47052.0,townhouse/condo,Harris County,No,Yes,None,[],[Slab],[],[composition],"[tile, vinyl]","[brick, stucco]","[balcony, partially fenced, storage]",[traditional],"[cash sale, conventional, fha, investor, va]",5.0,NaN,4.0,[CLEAR CREEK ISD],This is the perfect starter home! EVERYTHING U...


In [3]:
[_ for _ in raw_df.columns.values if "desc" in _]

['Private_pool_desc', 'general_desc']

In [5]:
raw_df = raw_df[["address", "address_key", "general_desc", "Private_pool_desc"]]
raw_df.sample(2)

,address,address_key,general_desc,Private_pool_desc
53,"118 Whites Lake Estates Dr, Highlands, TX 77562","118 Whites Lake Estates Dr, Highlands, TX 77562",Step into serenity living with this impeccably...,"Gunite, In Ground"
327,"6414 Buffalo Bend Ln, Richmond, TX 77469,","6414 Buffalo Bend Ln, Richmond, TX 77469","Brand new, energy-efficient home available NOW...",None


In [6]:
raw_df["general_desc"].isna().sum(), raw_df["Private_pool_desc"].isna().sum(), len(raw_df)

(1553, 189661, 211960)

In [7]:
tem = raw_df["general_desc"].apply(lambda x: len(x) if x is not None else 0)
tem[tem > 0].describe()

count    210406.0
mean        771.0
std         329.9
min           9.0
25%         538.0
50%         793.0
75%         993.0
max        4001.0
Name: general_desc, dtype: float64

In [8]:
tem = raw_df["Private_pool_desc"].apply(lambda x: len(x) if x is not None else 0)
tem[tem > 0].describe()

count    22299.0
mean        27.4
std         23.1
min          3.0
25%          9.0
50%         17.0
75%         37.0
max        166.0
Name: Private_pool_desc, dtype: float64

In [9]:
from transformers import pipeline
feature_extraction = pipeline('feature-extraction', model="roberta-base", tokenizer="roberta-base", device=0, truncation=True)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
cnt = 0 
general_desc_mean_features = []
general_desc_cls_features = []
for desc in tqdm(raw_df["general_desc"].values):
    if desc is None:
        cnt += 1
        general_desc_mean_features.append(np.zeros(768))
        general_desc_cls_features.append(np.zeros(768))
    else: 
        feature = feature_extraction(desc,  return_tensors="pt")
        general_desc_mean_features.append(np.mean(feature[0].clone().numpy(), axis=0))
        general_desc_cls_features.append(feature[0][0].clone().numpy())

raw_df["general_desc_roberta_mean_features"] = general_desc_mean_features
raw_df["general_desc_roberta_cls_features"] = general_desc_cls_features
raw_df[["address", "address_key", "general_desc_roberta_mean_features", "general_desc_roberta_cls_features"]].to_pickle("data/property_structured_12162024_general_desc_features.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 211960/211960 [1:01:22<00:00, 57.56it/s]


In [11]:
pool_desc_mean_features = []
pool_desc_cls_features = []
for desc in tqdm(raw_df["Private_pool_desc"].values):
    if desc is None:
        pool_desc_mean_features.append(np.zeros(768))
        pool_desc_cls_features.append(np.zeros(768))
    else: 
        feature = feature_extraction(desc,  return_tensors="pt")
        pool_desc_mean_features.append(np.mean(feature[0].clone().numpy(), axis=0))
        pool_desc_cls_features.append(feature[0][0].clone().numpy())
raw_df["pool_desc_roberta_mean_features"] = pool_desc_mean_features
raw_df["pool_desc_roberta_cls_features"] = pool_desc_cls_features
raw_df[["address", "address_key", "pool_desc_roberta_mean_features", "pool_desc_roberta_cls_features"]].to_pickle("data/property_structured_12162024_pool_desc_features.pkl")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 211960/211960 [03:43<00:00, 949.64it/s]
